# **Cats and Dogs Classify** - *ResNet*

***

## Preparação do ambiente

### Criação da pasta *data/*

Pasta que vai conter toda a **base de dados**.

In [ ]:
!test ! -d ../data && mkdir ../data

### Download do arquivo compactado de dados

caso os dados já tenham sido baixados na pasta *data/*, o download não será feito novamente.

In [ ]:
!test ! -d ../data/images && test ! -f ../data/images.tar.gz && wget -P ../data https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz

### Extração dos dados compactados para a pasta *data/images/*

a extração ocorrerá somente se o arquivo compactado de dados existir dentro da pasta *data/*.

In [ ]:
!test -f ../data/images.tar.gz && tar xf ../data/images.tar.gz -C ../data

### Remoção do arquivo compactado de dados da pasta *data/*

In [ ]:
!test -f ../data/images.tar.gz && rm ../data/images.tar.gz

### Bibliotecas utilizadas

In [ ]:
import os
import sys
import random
import numpy as np
from PIL import Image
import tensorflow as tf
from IPython.display import Image as display_image
from IPython.display import display

In [ ]:
sys.path.append('../')

### Importando Código Reutilizável 

In [ ]:
from utils.extract_data import create_dataframe, plot_samples
from utils.split_data import split_dataframe, create_train_generator, create_test_valid_generators
from utils.plot_heatmap import process_sample, predict_sample, get_last_conv_layer_name, make_gradcam_heatmap, save_and_display_gradcam

***

## Criação do *dataset*

### Setando a *seed*

In [ ]:
seed_value = 42

os.environ['PYTHONHASHSEED'] = str(seed_value)

random.seed(seed_value)

np.random.seed(seed_value)

tf.random.set_seed(seed_value)

### Criando *DataFrame* de dados

In [ ]:
dataset = create_dataframe('../data/images/')

dataset

### Plotando algumas amostras

In [ ]:
plot_samples(dataset)

***

## Divisão dos Dados em **Treino**, **Teste** e **Validação**

* 70% para treino
* 20% para validação
* 10% para teste

Dados serão divididos respeitando a proporção de quantidade de cada raça das duas classes a fim de evitar desbalanceamento.

In [ ]:
train_df, test_df, valid_df = split_dataframe(dataset)

In [ ]:
train_generator = create_train_generator(train_df, tf.keras.applications.resnet.preprocess_input)

In [ ]:
test_generator, valid_generator = create_test_valid_generators(test_df, valid_df, tf.keras.applications.resnet.preprocess_input)

***

## Construção e Treino do Modelo *Resnet*

### Definindo Constantes

In [ ]:
fig_size = 300
batch_size = 8
epochs = 10

### *ResNet50*

In [ ]:
# TODO: explicar cada parametro em cada etapa

input_shape = tf.keras.Input(shape = (fig_size, fig_size, 3))

resNet = tf.keras.applications.ResNet50(
    weights = 'imagenet',
    include_top = False,
    input_tensor = input_shape
)
resNet.trainable = False

maxPooling_layer = tf.keras.layers.GlobalMaxPooling2D()(resNet.output)
dense_layer = tf.keras.layers.Dense(256, activation = 'relu')(maxPooling_layer)
drouput_layer = tf.keras.layers.Dropout(0.2)(dense_layer)

output = tf.keras.layers.Dense(2, activation = 'softmax')(drouput_layer)

model = tf.keras.Model(resNet.input, output)

### Definindo *Callbacks*

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 4, min_delta = 0.01),
    tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_acc', factor = 0.2, patience = 4)
]

### Compilação do Modelo

In [ ]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = "adam",
    metrics = ["acc"],
)

### Treino do Modelo

In [ ]:
if not os.path.isfile('output/res_net.keras'):
    history = model.fit(
        train_generator,
        batch_size = batch_size,
        epochs = epochs,
        steps_per_epoch = int(np.ceil(train_generator.n / float(batch_size))),
        validation_data = valid_generator,
        validation_steps = int(np.ceil(valid_generator.n / float(batch_size))),
        validation_batch_size = batch_size,
        verbose = 1,
        callbacks = [callbacks]
    )

    if not os.path.exists('output'):
        os.makedirs('output')

    model.save('output/res_net.keras')

else:
    model = tf.keras.models.load_model('output/res_net.keras')

    print('Model loaded')

### Avaliando Modelo com a Base de Teste

In [ ]:
test_loss, test_acc = model.evaluate(
    test_generator,
    batch_size = batch_size,
    verbose = 0,
    steps = (np.ceil(test_generator.n / float(batch_size)))
)

test_acc

***

## Testes

In [ ]:
conv_last_layer_name = get_last_conv_layer_name(model)

conv_last_layer_name

### Teste 1

In [ ]:
sample = test_df.sample(1)['image_path'].values[0]

image_sample = Image.open(sample)

image_sample = image_sample.resize((fig_size, fig_size))

image_sample

In [ ]:
array_sample = process_sample(image_sample, fig_size, tf.keras.applications.resnet.preprocess_input)

predict, ypred = predict_sample(array_sample, train_generator, model)

predict

In [ ]:
print('Classe predita: ', predict)
print('probabilidade classe gato', ypred[0][0] * 100)
print('probabilidade classe cachorro', ypred[0][1] * 100)

In [ ]:
heatmap = make_gradcam_heatmap(array_sample, model, conv_last_layer_name, pred_index = np.argmax(ypred))

save_and_display_gradcam(sample, heatmap)

### Teste 2

In [ ]:
img_path = tf.keras.utils.get_file(origin = "https://sp-ao.shortpixel.ai/client/to_webp,q_glossy,ret_img,w_1200,h_675/https://vedovatipisos.com.br/wp-content/uploads/2015/12/ra%C3%A7as-de-cachorro-1200x675.jpg",
                     fname = 'dog_sample.jpg'
)

display(display_image(img_path))

array_sample = process_sample(Image.open(img_path), fig_size, tf.keras.applications.resnet.preprocess_input)

predict, ypred = predict_sample(array_sample, train_generator, model)

In [ ]:
print('Classe predita: ', predict)
print('probabilidade classe gato', ypred[0][0] * 100)
print('probabilidade classe cachorro', ypred[0][1] * 100)

In [ ]:
heatmap = make_gradcam_heatmap(array_sample, model, conv_last_layer_name, pred_index = np.argmax(ypred))

save_and_display_gradcam(img_path, heatmap)